**SORA - Stellar Occultation Reduction and Analysis**

## _prediction()_ and PredictionTable Object Class

The prediction module within SORA was created to predict stellar occultation, organize the prediction and plot the occultation maps. The documentation at <font color=blue>add link</font> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the prediction module. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

<font size="6">Index</font>

1. [Predicting stellar occultation](#section_1)
2. [Setting and/or modifying parameters](#section_2)
3. [Apparent Sidereal Time](#section_3)
4. [Ksi and Eta projection](#section_4)
5. [MPC observatories](#section_5)

<a id="section_1"></a>
<h3 id="section_1">1. Predicting stellar occultation</h3>

To predict a stellar occultation, the function _prediction()_ must be used. It will need an EphemKernel Object and the time interval for the search. The function will generate the ephemeris within the given interval (with a default  step of 60 seconds between positions, that may be changed by the user) and download the star coordinates from Vizier. For every star, it checks the closest ephemeris. An occultation is identified if the distance is smaller than the radius of the Earth plus the radius of the object (given in Ephem) plus the error of the ephemeris (multiplied by a sigma factor given by the user). In the next step, the coordinate of the star is propagated to date using proper motion and parallax. It results in the occultation parameters, such as the Closest Approach distance, the Position Angle, Velocity, etc.

The function returns a PredictionTable with all the information.

**All functions have their Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

In [1]:
from sora.prediction import prediction

In [2]:
prediction?

Signature:
prediction(
    time_beg,
    time_end,
    body=None,
    ephem=None,
    mag_lim=None,
    step=60,
    divs=1,
    sigma=1,
    radius=None,
    log=True,
)
Docstring:
Predicts stellar occultations

Parameters:
    time_beg (str,Time): Initial time for prediction (required).
    time_beg (str,Time): Final time for prediction (required).
    body* (Body, str): Object that will occult the stars. It must be a Body object or its
            name to search in the Small Body Database.
    ephem* (Ephem): object ephemeris. It must be an Ephemeris object.
    mag_lim (int,float): Faintest Gmag for search
    step (number): step, in seconds, of ephem times for search
    divs (int): number of regions the ephemeris will be splitted for better search of occultations
    sigma (number): ephemeris error sigma for search off-Earth.
    radius (number): The radius of the body. It is important if not defined in body or ephem.
    log (bool): To show what is being done at the moment.

* W

**Importing and defining the Ephemeris to be used.**

In [3]:
from sora.ephem import EphemKernel
eph_chariklo = EphemKernel(name='Chariklo', spkid='2010199', kernels=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'])

In [10]:
pred = prediction(eph_chariklo, time_beg='2017-06-20 00:00:00', time_end='2017-07-01 00:00:00', mag_lim=18, divs=3)

TypeError: prediction() got multiple values for argument 'time_beg'

<a id="section_2"></a>
<h3 id="section_2">2. The PredictionTable</h3>

The resulting variable of the prediction function is a PredictionTable Object. The PredictionTable Class was created to have all the information about the occultations predicted, plot occultation maps and save the table in other formats.

**The PredictionTable is a class that inherits all methods from Astropy.table.Table. All the functions present in Astropy Table should work for PredictionTable.**

More information about it at https://docs.astropy.org/en/stable/table/

**if the user asks to print the prediction, a summary of the table is shown.**

In [12]:
print(pred)

NameError: name 'pred' is not defined

**to print the complete table, we must use:**

In [13]:
pred.pprint_all()

NameError: name 'pred' is not defined

*M-G-T stands for Moon-Geocenter-Target, the on-sky angle between the star and the moon*

*S-G-T stands for Sun-Geocenter-Target, the on-sky angle between the star and the sun*

**The PredictionTable can be exported to PRAIA format file**

In this case, some information, such as the Gaia-DR2 Source ID, S-G-T and M-G-T, are lost.

In [14]:
pred.to_praia?

Object `pred.to_praia` not found.


In [15]:
pred.to_praia('output/Chariklo_occs.table')
# Check your folder named output

NameError: name 'pred' is not defined

**The PredictionTable can also generate the files in the Occult Watcher feed format**

These are two files, named 'LOG.dat' and 'tableOccult_update.txt'. The occult watcher designation for the object must be informed. For instance, for the satellite Himalia, the designation is "P5M06". If mode='append' the prediction will be appended to an existing file. If mode='restart', the files will overwritten.

In [16]:
pred.to_ow?

Object `pred.to_ow` not found.


In [17]:
pred.to_ow('1997CU26')

NameError: name 'pred' is not defined

**Using the functions from Astropy Table, the data can be exported to many other formats using the funtion 'write'**

The formats allowed can be seen in https://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers. This includes latex and csv.

In [18]:
pred.write('output/prediction.tex')
# Check your folder named output

NameError: name 'pred' is not defined

<a id="section_3"></a>
<h3 id="section_3">3. Using PredictionTable</h3>

The PredictionTable was not created to be instantiated directly by the user. It was designed to be an insteresting output object from the prediction function. However, a PredictionTable can be instantiated from a PRAIA occultation table. For this, a PredictionTable method must be called directly as shown below.

In [19]:
from sora.prediction import PredictionTable

In [20]:
PredictionTable.from_praia?

Signature: PredictionTable.from_praia(filename, name, **kwargs)
Docstring:
Creates a PredictionTable Table reading from a PRAIA table

Parameters:
    filename (str): path to the PRAIA table file.
    name (str): Name of the Object of the prediction.
    radius (int,float): Object radius, in km. (not required)
        If not given it's searched in online database.
        If not found online, the default is set to zero.

Returns:
    A PredictionTable
File:      ~/miniconda3/lib/python3.8/site-packages/sora/prediction.py
Type:      method


In [21]:
pred_1 = PredictionTable.from_praia(filename='output/Chariklo_occs.table', name='Chariklo')

OSError: File output/Chariklo_occs.table not found

In [22]:
pred_1.pprint_all()

NameError: name 'pred_1' is not defined

To remove some occultations the User can use two functions. The first is the _PredictionTable.remove_occ()_.

In [23]:
pred_1.remove_occ?

Object `pred_1.remove_occ` not found.


In [24]:
pred_1.remove_occ('2017-06-21 09:57')

NameError: name 'pred_1' is not defined

In [25]:
pred_1.pprint_all()

NameError: name 'pred_1' is not defined

The second function is the _PredictionTable.keep_from_selected_images()_ this function will check the names in the saved images and eliminate the occultations without the map. This functions allows that after the user deleted the maps of the unwanted occultation this function will eliminate the respective rows in the _PredictionTable_.  

In [26]:
pred_1.keep_from_selected_images?

Object `pred_1.keep_from_selected_images` not found.


<a id="section_4"></a>
<h3 id="section_4">4. Plotting occultation maps</h3>

SORA is also able to generate an occultation map. The only required parameters are the occultation parameters. The function also has many inputs to configure the plot.

The first time the function is called, cartopy will download some data referring to the features presented in the map, such as the country and state division, lakes, rivers, etc.

In [27]:
from sora.prediction import plot_occ_map

In [28]:
plot_occ_map?

Signature:
plot_occ_map(
    name,
    radius,
    coord,
    time,
    ca,
    pa,
    vel,
    dist,
    mag=0,
    longi=0,
    **kwargs,
)
Docstring:
Plots map of the occultation

Parameters:
    Required params:
    name (str): Name of the object
    radius (int, float): radius of the object (km)
    coord (str, SkyCoord): Coordinate of the star
        ("hh mm ss.sss dd mm ss.sss" or "hh.hhhhhhhh dd.dddddddd")
    time (str, Time): Instant of Closest Approach (iso or isot format)
    ca (int, float): Closest Approach Distance (arcsec)
    pa (int, float): Position Angle at C/A (deg)
    vel (int, vel): Velocity of the event (km/s)
    dist (int, float): Object distance at C/A (AU)

    Not required params (only printed in label):
    mag (int,float): Mag* = Normalized magnitude to vel=20km/s
    longi (int,float): East longitude of sub-planet point, deg, positive towards East

    Map configuration:
    nameimg (str): Change the name of the imaged saved.
    path (str): Path to a

**To make it easier to plot the predictions, the** _PredictionTable.plot_occ_map()_ **function can be called which automatically fills the required parameters.**

To plot the map for all the predictions


In [29]:
pred.plot_occ_map(path='output/')

NameError: name 'pred' is not defined

To plot the map for only one prediction, just give a item to the PredictionTable object.

In [30]:
pred[0].plot_occ_map(path='output/')

NameError: name 'pred' is not defined

**The PredictionTable can also be plotted by giving the date of the occultation. All the occultations that matches the date will be plotted.**

The date can be as constrained as the user wants, and must match the text that appears in the 'Epoch' column.

In [31]:
pred['2017-06-26'].plot_occ_map()

NameError: name 'pred' is not defined

<a id="section_5"></a>
<h3 id="section_5">5. Occultation parameters</h3>

Finally, a function is implemented in the prediction module which calculates the occultation parameters. For this function, it must be passed a Star objetct, an Ephemeris object (it can be any of the Ephemeris classes) and a time. The time does not need to be precise, but it must be close within 10 minutes from the Closest Approach.

In [32]:
from sora.prediction import occ_params

In [33]:
occ_params?

Signature: occ_params(star, ephem, time, n_recursions=5, max_tdiff=None)
Docstring:
Calculates the parameters of the occultation, as instant, CA, PA.

Parameters:
    star (Star): The coordinate of the star in the same reference frame as the ephemeris.
        It must be a Star object.
    ephem (Ephem): object ephemeris. It must be an Ephemeris object.

Returns:
    instant of CA (Time): Instant of Closest Approach
    CA (arcsec): Distance of Closest Approach
    PA (deg): Position Angle at Closest Approach
    vel (km/s): Velocity of the occultation
    dist (AU): the object geocentric distance.
    n_recursions (int): The number of attempts to try obtain prediction parameters
        in case the event is outside the previous range of time. Default=5
    max_tdiff (int): Maximum difference from given time it will attempt to identify
        the occultation, in minutes. If given, 'n_recursions' is ignored. Default=None.
File:      ~/miniconda3/lib/python3.8/site-packages/sora/predict

In [34]:
from sora.star import Star
s = Star(code='6760225712991422208', log=False)

In [35]:
tca, ca, pa, vel, dist = occ_params(s, eph_chariklo, time='2017-06-26 20:40')
print('Time of the CA:   {} UTC'.format(tca))
print('Closest Approuch: {:.3f}'.format(ca))
print('Position Angle:   {:.3f}'.format(pa))
print('Shadow velocity:  {:.3f}'.format(vel))
print('Object distance:  {:.5f}'.format(dist))

Time of the CA:   2017-06-26 20:46:07.140 UTC
Closest Approuch: 0.199 arcsec
Position Angle:   0.511 deg
Shadow velocity:  -22.333 km / s
Object distance:  14.65229 AU


**This Jupyter-Notebook was designed as a tutorial for how to work with the** _prediction()_ **functions and PredictionTable Object Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

*The End.*